In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# client = pd.read_csv('cliente_tabla.csv') 
# product = pd.read_csv('producto_tabla.csv')
# townst = pd.read_csv('town_state.csv')
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
btrain = pd.read_csv('bimbo_train_baru.csv')
btest = pd.read_csv('bimbo_test_baru.csv')

In [3]:
btrain = btrain.drop(['Unnamed: 0'],axis=1)
btest = btest.drop(['Unnamed: 0'],axis=1)

In [4]:
idx = btrain.groupby('Town').sum().sort_values(by='venta_hoy', ascending=False).head(2).index

In [5]:
btraintop2 = btrain[(btrain['Town'].isin(idx))]
btesttop2 = btest[(btest['Town'].isin(idx))]

In [6]:
btraintop2.head()

,semana,agencia_id,canal_id,ruta_sak,cliente_id,producto_id,venta_uni_hoy,venta_hoy,dev_uni_proxima,dev_proxima,demanda_uni_equil,unit_price,stok,NombreCliente,Town,NombreProducto
0,4,1379,2,7017,26988,30236,24,748.8,0.0,0.0,24,31.2,24.0,MEGA COMERCIAL MEXICANA MAZATLAN,2330 TLAQUEPARQUE,Barritas Fresa KC 8p 600g CU MLA 30236
1,5,1379,2,7017,26988,30236,24,748.8,0.0,0.0,24,31.2,24.0,MEGA COMERCIAL MEXICANA MAZATLAN,2330 TLAQUEPARQUE,Barritas Fresa KC 8p 600g CU MLA 30236
2,7,1379,2,7017,26988,30236,48,1497.6,0.0,0.0,48,31.2,48.0,MEGA COMERCIAL MEXICANA MAZATLAN,2330 TLAQUEPARQUE,Barritas Fresa KC 8p 600g CU MLA 30236
3,6,1379,2,7017,26988,30236,24,748.8,0.0,0.0,24,31.2,24.0,MEGA COMERCIAL MEXICANA MAZATLAN,2330 TLAQUEPARQUE,Barritas Fresa KC 8p 600g CU MLA 30236
4,8,1379,2,7017,26988,30236,24,748.8,0.0,0.0,24,31.2,24.0,MEGA COMERCIAL MEXICANA MAZATLAN,2330 TLAQUEPARQUE,Barritas Fresa KC 8p 600g CU MLA 30236


In [7]:
predict = btraintop2.groupby(['cliente_id','producto_id','ruta_sak']).mean()[['demanda_uni_equil']]
predict.columns = ['mean_all_prior']

In [8]:
predict = predict.reset_index()

In [9]:
predict.head()

,cliente_id,producto_id,ruta_sak,mean_all_prior
0,60,32336,7329,192.0
1,60,32336,7773,288.0
2,60,32336,7774,288.0
3,60,32336,7777,192.0
4,60,32336,7779,384.0


In [10]:
predict3 = btraintop2[btraintop2['semana'].isin(range(6,9))]
predict3 = predict3.groupby(['cliente_id','producto_id','ruta_sak']).mean()[['demanda_uni_equil']]
predict3.columns = ['3-weeks moving avg']

In [11]:
predict3 = predict3.reset_index()
predict3.head()

,cliente_id,producto_id,ruta_sak,3-weeks moving avg
0,60,32336,7329,192.0
1,60,32336,7773,320.0
2,60,32336,7774,192.0
3,60,32336,7777,192.0
4,60,32336,7779,384.0


In [12]:
predict2 = btraintop2[btraintop2['semana']==8][['cliente_id','producto_id','ruta_sak','demanda_uni_equil']]
predict2.columns = ['cliente_id','producto_id','ruta_sak','target']

In [13]:
predict2.head()

,cliente_id,producto_id,ruta_sak,target
4,26988,30236,7017,24
10,9733973,30236,1603,48
16,19635,30236,1610,144
22,26945,30236,1617,72
28,591026,30236,1642,72


In [14]:
npredict = pd.merge(predict,predict3, on=['cliente_id','producto_id','ruta_sak'])

In [15]:
npredict.head()

,cliente_id,producto_id,ruta_sak,mean_all_prior,3-weeks moving avg
0,60,32336,7329,192.0,192.0
1,60,32336,7773,288.0,320.0
2,60,32336,7774,288.0,192.0
3,60,32336,7777,192.0,192.0
4,60,32336,7779,384.0,384.0


In [16]:
newpredict = pd.merge(npredict,predict2,on=['cliente_id','producto_id','ruta_sak'])

In [17]:
newpredict.head()

,cliente_id,producto_id,ruta_sak,mean_all_prior,3-weeks moving avg,target
0,60,32336,7329,192.0,192.0,192
1,60,32336,7773,288.0,320.0,576
2,60,32336,7777,192.0,192.0,192
3,60,32336,7779,384.0,384.0,384
4,60,32336,7783,288.0,384.0,384


In [18]:
neopredict = pd.merge(newpredict,btesttop2[['cliente_id','producto_id','ruta_sak','demanda_uni_equil']],
                      on=['cliente_id','producto_id','ruta_sak'],how='left')

In [19]:
neopredict = neopredict[neopredict['demanda_uni_equil'].notnull()]
neopredict.head()

,cliente_id,producto_id,ruta_sak,mean_all_prior,3-weeks moving avg,target,demanda_uni_equil
1,60,32336,7773,288.0,320.0,576,960.0
3,60,32336,7779,384.0,384.0,384,1344.0
6,101,31200,7334,60.0,56.0,96,120.0
7,107,48021,3083,14.0,12.0,12,12.0
9,19404,30236,1652,56.0,24.0,24,24.0


In [20]:
neopredict = round(neopredict,0).astype(int)

In [21]:
neopredict.head()

,cliente_id,producto_id,ruta_sak,mean_all_prior,3-weeks moving avg,target,demanda_uni_equil
1,60,32336,7773,288,320,576,960
3,60,32336,7779,384,384,384,1344
6,101,31200,7334,60,56,96,120
7,107,48021,3083,14,12,12,12
9,19404,30236,1652,56,24,24,24


In [28]:
neopredict0 = pd.merge(newpredict,btesttop2[['cliente_id','producto_id','ruta_sak','demanda_uni_equil']],
                      on=['cliente_id','producto_id','ruta_sak'],how='right')
neopredict1 = pd.merge(newpredict,btesttop2[['cliente_id','producto_id','ruta_sak','demanda_uni_equil']],
                      on=['cliente_id','producto_id','ruta_sak'],how='left')

In [29]:
databaru = neopredict1[neopredict1['demanda_uni_equil'].isnull()]
databaru0 = neopredict0[neopredict0['mean_all_prior'].isnull()]

In [30]:
databaru0.head()

,cliente_id,producto_id,ruta_sak,mean_all_prior,3-weeks moving avg,target,demanda_uni_equil
1756,9838234,30236,4178,NaN,NaN,NaN,72
1757,9838234,30235,4182,NaN,NaN,NaN,72
1758,886620,30235,4192,NaN,NaN,NaN,72
1759,886620,30235,4194,NaN,NaN,NaN,48
1760,886620,30235,4201,NaN,NaN,NaN,24


In [31]:
databaru1 = pd.merge(databaru,btraintop2[['producto_id','NombreProducto']],on='producto_id')
produklamanonstok = pd.DataFrame(databaru1['NombreProducto'].unique(), columns=['Nama Produk'])

In [32]:
databaru2 = pd.merge(databaru0,btraintop2[['producto_id','NombreProducto']],on='producto_id')
produkbaru = pd.DataFrame(databaru2['NombreProducto'].unique(), columns=['Nama Produk'])

In [33]:
print('Produk yang sebelumnya di stok tapi sudah enggak lagi')
print(produklamanonstok)
print('--'*25)
print('Produk baru di stok')
print(produkbaru)

Produk yang sebelumnya di stok tapi sudah enggak lagi
                                     Nama Produk
0                  Salmas 36p 612g CUB SAN 32336
1                Canelitas 14p 840g CU MLA 36978
2             Lata Butter Danesas 400g GBI 48021
3             Tostada Clasica 240g CJM SAN 31187
4          Barritas Pina KC 8p 600g CU MLA 30235
5                  Principe KC 392g CU MLA 35526
6     Triki Trakes Tubo 20p 174g SP CU MLA 36948
7         Tostada Briobalance 240g CJM SAN 31188
8                      Salmas 414g CJM SAN 31200
9                      Nachos 250g CJM SAN 31201
10   Tost Maiz Nopal sn Grasa 240g CJM SAN 32335
11                     Besos Nuez 375g GBI 33643
12                   Surtido Rico 770g GBI 48001
13                Surtido Fino 375g Sp GBI 48002
14            Totopos de Maiz 200g CJM SAN 33382
15       Sandwich Clasico Deli 169g CU LON 35897
16        Sandwich Doble Fibra 187g CU LON 42083
17             Sandwich Linaza 179g CU LON 42086
18  Sandwich Cl

In [35]:
produklamanonstok

,Nama Produk
0,Salmas 36p 612g CUB SAN 32336
1,Canelitas 14p 840g CU MLA 36978
2,Lata Butter Danesas 400g GBI 48021
3,Tostada Clasica 240g CJM SAN 31187
4,Barritas Pina KC 8p 600g CU MLA 30235
5,Principe KC 392g CU MLA 35526
6,Triki Trakes Tubo 20p 174g SP CU MLA 36948
7,Tostada Briobalance 240g CJM SAN 31188
8,Salmas 414g CJM SAN 31200
9,Nachos 250g CJM SAN 31201


In [ ]:
neopredict.isnull().sum()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
RMSE_MA = np.sqrt(mean_squared_error(neopredict['demanda_uni_equil'], neopredict['3-weeks moving avg']))
RMSE_AP = np.sqrt(mean_squared_error(neopredict['demanda_uni_equil'], neopredict['mean_all_prior']))
RMSE_P = np.sqrt(mean_squared_error(neopredict['demanda_uni_equil'], neopredict['target']))

In [ ]:
print(RMSE_MA)
print(RMSE_AP)
print(RMSE_P)

In [ ]:
neopredict['Ensemble']=(0.15*neopredict['target']+0.15*neopredict['mean_all_prior']+0.7*neopredict['3-weeks moving avg'])

In [ ]:
RMSE_E = np.sqrt(mean_squared_error(neopredict['demanda_uni_equil'], neopredict['Ensemble']))

In [ ]:
RMSE_E